In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import json

import lightgbm as lgb
from sklearn.model_selection import GroupKFold

print("Lightgbm version:", lgb.__version__)

import optuna
from optuna.visualization import (
    plot_edf
    , plot_optimization_history
    , plot_parallel_coordin
    ate
    , plot_param_importances
    , plot_slice
)

print("Optuna version:", optuna.__version__)

# local modules
import sys
sys.path.append("../src")
from preproc import process_train_data

***
### load and preprocess data

In [ ]:
# define some paths
path_raw = Path("../data/raw")
path_processed = Path("../data/processed")
path_results = Path("../data/results")

# load data
df_train = pd.read_csv(path_raw / "train.csv")
df_test = pd.read_csv(path_raw / "test.csv")

df_train

In [ ]:
# Load the feature selection results
with open('../feat_selection/select_optuna_lgbm.json', 'r') as f:
    feature_selection = json.load(f)

# Extract the selected features
numerical_cols = feature_selection.get('numerical', [])
categorical_cols = feature_selection.get('categorical', [])

print("Numerical features:", len(numerical_cols))
print("Categorical features:", len(categorical_cols))

In [ ]:
df_train, numerical_cols, categorical_cols, encoder, scaler = process_train_data(
    df_train,
    scale=False,
    numerical_cols=numerical_cols,
    categorical_cols=categorical_cols
)

# Print the results
print("Numerical Columns:", len(numerical_cols))
print("Categorical Columns:", len(categorical_cols))

***
### optimization



In [5]:
# Define the number of folds for cross-validation
num_folds = 5

# Shuffle the train dataset
df_train = df_train.sample(frac=1, random_state=2113).reset_index(drop=True)

# Define the column for stratified or group k-fold
groups_col = 'GameRulesetName'
gkf = GroupKFold(n_splits=num_folds)
split_list = list(gkf.split(df_train, groups=df_train[groups_col]))

In [6]:
def train_and_score(params):
    _params = params.copy()
    num_iterations = _params.pop('num_iterations')

    target = 'utility_agent1'
    oof_scores = []

    # Perform cross-validation
    for _, (train_index, val_index) in enumerate(split_list, 1):
        # Split the data
        X_train, X_val = df_train.iloc[train_index], df_train.iloc[val_index]
        y_train, y_val = X_train[target], X_val[target]
        
        # Create LightGBM datasets
        train_data = lgb.Dataset(
            data=X_train[numerical_cols + categorical_cols],
            label=y_train,
            categorical_feature=categorical_cols,
            free_raw_data=True
        )

        # Train the model
        model = lgb.train(
            _params,
            train_data,
            num_boost_round=num_iterations,
        )

        # predict on validation set
        y_pred = model.predict(X_val[numerical_cols + categorical_cols])
        y_pred = np.clip(y_pred, -1, 1)
        
        # Compute RMSE on scaled values
        rmse = np.sqrt(np.mean((y_pred - y_val) ** 2))
        oof_scores.append(rmse)

    return np.mean(oof_scores)


In [7]:
# Define the parameters
fixed_params = {
    'objective': "regression",
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'bagging_freq': 1,
    'verbose': -1,
    'seed': 2112,
}

def objective(trial):
    # params sampling
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 31, 255),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 100.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 100.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'num_iterations': trial.suggest_int('num_iterations', 1000, 5000),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 100, 1000),
        'min_data_per_group': trial.suggest_int('min_data_per_group', 100, 1000),
        'max_cat_threshold': trial.suggest_int('max_cat_threshold', 32, 512),
        'cat_l2': trial.suggest_float('cat_l2', 1e-8, 100.0, log=True),
        **fixed_params
    }
    
    # Call the train_and_score function
    score = train_and_score(params)
    
    return score

In [ ]:
do_optimize = True
timeout = 3600 * 48

study = optuna.create_study(
    study_name="lightgbm.db",
    direction='minimize',
    storage='sqlite:///lightgbm.db',
    load_if_exists=True,
)

if do_optimize:

    study.optimize(
        objective, 
        n_trials=5000, 
        timeout=timeout,
        n_jobs=1, 
        gc_after_trial=True,
    ) 

In [ ]:
study.trials_dataframe().sort_values("value", ascending=True).head(20)

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)


In [ ]:
plot_edf(study)


In [ ]:
plot_parallel_coordinate(study)


In [ ]:
best_params = dict(study.best_params)
best_params = {**fixed_params, **best_params}
best_params

***